In [1]:
import pandas as pd
import os
from predictionguard import PredictionGuard
from concurrent.futures import ThreadPoolExecutor
import time


# Set your API key
api_key = os.getenv("PREDICTIONGUARD_API_KEY", "Oq62vYfSJRwjnFQcUnJy5PM3SRVejYtJCXWSxnfv")

# Initialize the PredictionGuard client
client = PredictionGuard(api_key=api_key)

# System content for the chatbot behavior based on your refined prompt
system_message = {
    "role": "system",
    "content": (
        "You are a language identification and translation system. Your task is to process a message and return it in the following way:\n"
        "1. If the message is entirely in English, return the message exactly as is.\n"
        "2. If the message is in both English and another language:\n"
        "    - If the foreign language part is a direct translation of the English text, return only the English.\n"
        "    - If the foreign language contains information that is not a direct translation, translate the entire message to English and return it.\n"
        "3. If the message is entirely in a foreign language, translate it into English and return it.\n"
        "4. If a message seems to be unrelated to reviews, is spam, or contains malicious content, output SPAM\n"
        "Do not provide any explanations or contextual information. Only return the final processed message."
    )
}

# Input and output CSV file paths
csv_file = 'beer_reviews.csv'
output_file = 'translated_messages.csv'

# Read the CSV file using pandas
df = pd.read_csv(csv_file)

# Group the DataFrame by the beer name (first column) and take the first 100 reviews for each beer
df_100 = df.groupby(df.columns[0]).head(100)


# Function to process each message using PredictionGuard API
def process_message(row):
    try:
        user_message = row['message']  # Adjust column name as per your CSV
        
        # Prepare the messages list for the chatbot
        messages = [
            system_message,
            {
                "role": "user",
                "content": f"Do not provide any explanations or contextual information. Only return the final processed message for : {user_message}"
            }
        ]
        
        # Send the message to the PredictionGuard API
        result = client.chat.completions.create(
            model="Hermes-3-Llama-3.1-8B",
            messages=messages
        )
        
        # Extract the chatbot's response
        response = result['choices'][0]['message']['content']
        time.sleep(0.2)
        return response
    except Exception as e:
        return f"Error: {str(e)}"

# Use ThreadPoolExecutor to process messages concurrently
with ThreadPoolExecutor(max_workers=10) as executor:
    # Apply the translation function to each row and store the result in a new column
    df_100['translated_messages'] = list(executor.map(process_message, [row for _, row in df_100.iterrows()]))

# Save the updated DataFrame with translated messages to a new CSV file
df_100.to_csv(output_file, index=False)

print(f"Translated messages have been saved to {output_file}")


ValueError: Could not make prediction. 